In [66]:
import pandas as pd


In [67]:
df = pd.read_csv('BTCUSDT_5min_from_2019.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])  # 把 timestamp 轉為 datetime 格式
df.set_index('timestamp', inplace=True) 


# 每小時 00 分 open 價格（也作為 hour_k_open）
df["hour_00_open"] = df[df.index.minute == 0]["open"]
df["hour_k_open"] = df["hour_00_open"]

# 每小時 15 分 open 價格
df["hour_15_open"] = df[df.index.minute == 15]["open"]

# 每小時 15 分 open 價格
df["hour_30_open"] = df[df.index.minute == 30]["open"]

# 每小時 45 分 open 價格
df["hour_45_open"] = df[df.index.minute == 45]["open"]

# 每小時 55 分 close 價格（作為 hour_k_close）
df["hour_k_close"] = df[df.index.minute == 55]["close"]

#過濾掉前後沒有開收的資料，沒有完整一小時的資料。
# 3. 篩選完整區段
# df_filtered = df.loc['2020-01-01 18:00:00':'2025-06-29 12:55:00'].copy()  # 全历史数据范围
# df_filtered = df.loc['2024-06-29 18:00:00':'2025-06-29 12:55:00'].copy()  # 一年
# df_filtered = df.loc['2024-12-29 18:00:00':'2025-06-29 12:55:00'].copy()  # 半年
# df_filtered = df.loc['2025-03-29 18:00:00':'2025-06-29 12:55:00'].copy()  # 三月
# df_filtered = df.loc['2025-05-29 18:00:00':'2025-06-29 12:55:00'].copy()  # 一月
df_filtered = df.loc['2025-06-22 18:00:00':'2025-06-29 12:55:00'].copy()  # 一周

In [68]:
# 4. forward-fill 補值（每小時最多 12 根 5 分鐘 K 線）
df_filtered["hour_00_open"] = df_filtered["hour_00_open"].ffill(limit=11)
df_filtered["hour_k_open"] = df_filtered["hour_k_open"].ffill(limit=11)
df_filtered["hour_15_open"] = df_filtered["hour_15_open"].ffill(limit=11)
df_filtered["hour_30_open"] = df_filtered["hour_30_open"].ffill(limit=11)
df_filtered["hour_45_open"] = df_filtered["hour_45_open"].ffill(limit=11)
df_filtered["hour_k_close"] = df_filtered["hour_k_close"].ffill(limit=11)

In [69]:
# 5. 僅保留每小時的最後一根（55分）
df_hourly = df_filtered[df_filtered.index.minute == 55].copy()

# 6. 條件判斷與機率計算
total = len(df_hourly)

cond_up_15 = (df_hourly["hour_15_open"] - df_hourly["hour_00_open"] > 0) & (df_hourly["hour_k_close"] > df_hourly["hour_k_open"])
cond_down_15 = (df_hourly["hour_15_open"] - df_hourly["hour_00_open"] < 0) & (df_hourly["hour_k_close"] < df_hourly["hour_k_open"])

success_count_15 = (cond_up_15 | cond_down_15).sum()
probability_15 = success_count_15 / total

cond_up_30 = (df_hourly["hour_30_open"] - df_hourly["hour_00_open"] > 0) & (df_hourly["hour_k_close"] > df_hourly["hour_k_open"])
cond_down_30 = (df_hourly["hour_30_open"] - df_hourly["hour_00_open"] < 0) & (df_hourly["hour_k_close"] < df_hourly["hour_k_open"])

success_count_30 = (cond_up_30 | cond_down_30).sum()
probability_30 = success_count_30 / total

cond_up_45 = (df_hourly["hour_45_open"] - df_hourly["hour_00_open"] > 0) & (df_hourly["hour_k_close"] > df_hourly["hour_k_open"])
cond_down_45 = (df_hourly["hour_45_open"] - df_hourly["hour_00_open"] < 0) & (df_hourly["hour_k_close"] < df_hourly["hour_k_open"])

success_count_45 = (cond_up_45 | cond_down_45).sum()
probability_45 = success_count_45 / total

# 7. 輸出結果
print(f"總次數: {total}\n")

print(f"15分條件成立次數: {success_count_15}")
print(f"15分機率: {probability_15:.4%}\n")

print(f"30分條件成立次數: {success_count_30}")
print(f"30分機率: {probability_30:.4%}\n")

print(f"45分條件成立次數: {success_count_45}")
print(f"45分機率: {probability_45:.4%}")

總次數: 163

15分條件成立次數: 110
15分機率: 67.4847%

30分條件成立次數: 117
30分機率: 71.7791%

45分條件成立次數: 138
45分機率: 84.6626%
